In [1]:
import chromadb
import torch
from sentence_transformers import SentenceTransformer
import json

In [2]:
class BGEEmbeddingFunction():
    def __init__(self):
        device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
        print(f"Using device: {device}")
        self.device = device
        self.model = SentenceTransformer("BAAI/bge-large-en", device=device)
    
    def __call__(self, input: list[str]) -> list[list[float]]:
        if isinstance(input, str):
            input = [input]
        embeddings = self.model.encode(input, normalize_embeddings=True)
        return embeddings.tolist()
    
def init_embedding_model():
    return BGEEmbeddingFunction()

embedding_function = init_embedding_model()

Using device: mps


In [13]:
from chromadb.config import Settings

def init_chroma(embedding_function):
    # Create a persistent client
    client = chromadb.PersistentClient(
        path="./chroma_db",
        settings=Settings(
            anonymized_telemetry=False,
            allow_reset=True,
            persist_directory="./chroma_db",
        )
    )
    
    # Create or get collection
    collection = client.get_or_create_collection(
        name="conversations",
        embedding_function=embedding_function,
        metadata={"description": "Conversation QA pairs"}
    )
    
    return collection

collection = init_chroma(embedding_function)

In [14]:
def query_example(collection, query_text, n_results=5):
    results = collection.query(
        query_texts=[query_text],
        n_results=n_results
    )
    return results

In [19]:
query = "Please explain the level of inclusion of the planning process."
results = query_example(collection, query, 1)
print("\nExample query results:")
print(json.dumps(results['documents'][0][0], indent=2))


Example query results:
"Question: Please explain the level of inclusion of the planning process.\nAnswer: I'm sorry, but based on the given context, it is unclear how your question is related to the topic. Could you please give me more information or context about what you are referring to as \"the planning process\"? This will help me provide you with a more helpful and detailed answer."
